In [1]:
import requests
from alphacast import Alphacast
import pandas as pd
import time
from clave_api_alphacast import api_key

In [2]:
alphacast = Alphacast(api_key)

In [3]:
tc_serie_alpha = alphacast.datasets.dataset(5288).download_data("pandas")
tc_serie_alpha

,Date,country,ADR GGAL,BLUE,Dolar CCL,Dolar MEP,Dolar Mayorista,Dolar Oficial,GGAL,Merval,...,Brecha CCL - 30d_change,Brecha Blue - 30d_change,Dolar Oficial - 30d_change,Dolar Solidario,Dolar Solidario - 30d_running_av,Dolar Solidario - 30d_running_av_mom_change,Dolar Solidario - 30d_change,Dolar Solidario - usd_daily,Dolar Solidario - 7d_running_av,Dolar Solidario - 1d_change
0,1992-01-02,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-01-03,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992-01-06,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992-01-07,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992-01-08,Argentina,NaN,NaN,NaN,NaN,NaN,0.991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8551,2023-12-25,Argentina,NaN,NaN,941.96,NaN,NaN,NaN,NaN,NaN,...,-1.000000,-1.000000,1.247802,NaN,1126.450667,0.491384,0.386953,NaN,1318.514286,NaN
8552,2023-12-26,Argentina,17.74,970.0,901.25,939.70,806.8833,843.550,1664.15,908280.32,...,-0.926340,-0.881994,1.247190,1322.8,1138.839000,0.493277,0.390738,1.643740,1319.371429,NaN
8553,2023-12-27,Argentina,17.56,975.0,870.97,932.59,807.4250,843.510,1600.35,890877.24,...,-0.943405,-0.877036,1.239268,1323.6,1151.126500,0.494756,0.386005,1.640454,1320.228571,0.060478
8554,2023-12-28,Argentina,17.30,975.0,945.02,950.24,807.9750,843.510,1664.65,911278.28,...,-0.875390,-0.867396,1.234168,1324.4,1163.398167,0.496104,0.384993,1.640327,1321.085714,0.060441


In [4]:
df = pd.read_excel('../output/ITCRM historico.xlsx',sheet_name="tc_equilibrio")
tc_read_df = pd.read_excel('../output/Tc a precios de hoy.xlsx')[['Período','dolar','inflacion','tc_real_hoy']]

In [5]:
tc_serie_alpha = tc_serie_alpha.set_index("Date")
tc_serie_alpha.index = pd.to_datetime(tc_serie_alpha.index, format = "%Y-%m-%d")
tc_serie_alpha = tc_serie_alpha[["BLUE", "Dolar CCL", "Dolar MEP", "Dolar Solidario"]]
df = df.set_index("Período")

In [6]:
df_brechas = df.merge(tc_serie_alpha, how = "outer", left_index = True, right_index = True)
df_brechas["blue-equilibrio"] = (df_brechas["BLUE"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]
df_brechas["ccl-equilibrio"] = (df_brechas["Dolar CCL"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]
df_brechas["mep-equilibrio"] = (df_brechas["Dolar MEP"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]
df_brechas["solidario-equilibrio"] = (df_brechas["Dolar Solidario"] - df_brechas["tc_equilibrio"]) / df_brechas["tc_equilibrio"]

In [7]:
df

,tc_equilibrio,tc_oficial_mayorista,brecha
Período,,,
1997-11-24,1.924512,1.00,0.924512
1997-11-25,1.927250,1.00,0.927250
1997-11-26,1.932110,1.00,0.932110
1997-11-27,1.933234,1.00,0.933234
1997-11-28,1.932836,1.00,0.932836
...,...,...,...
2023-12-25,795.173732,804.75,-0.011900
2023-12-26,797.045357,806.85,-0.012152
2023-12-27,799.086641,807.40,-0.010296


In [8]:
df_brechas

,tc_equilibrio,tc_oficial_mayorista,brecha,BLUE,Dolar CCL,Dolar MEP,Dolar Solidario,blue-equilibrio,ccl-equilibrio,mep-equilibrio,solidario-equilibrio
1992-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-25,795.173732,804.75,-0.011900,NaN,941.96,NaN,NaN,NaN,0.184596,NaN,NaN
2023-12-26,797.045357,806.85,-0.012152,970.0,901.25,939.70,1322.8,0.216995,0.130739,0.178979,0.659630
2023-12-27,799.086641,807.40,-0.010296,975.0,870.97,932.59,1323.6,0.220143,0.089957,0.167070,0.656391
2023-12-28,803.285821,807.95,-0.005773,975.0,945.02,950.24,1324.4,0.213765,0.176443,0.182941,0.648728


In [9]:
df_brechas.index[-1].strftime("%d/%m/%Y")

'29/12/2023'

In [10]:
import plotly.graph_objects as go

fig = go.Figure()
df_brechas = df_brechas["2011":]
ultima_fecha = df_brechas.index[-1].strftime("%d/%m/%Y")
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["solidario-equilibrio"], name = "Solidario"))
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["blue-equilibrio"], name = "Blue"))
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["ccl-equilibrio"], name = "CCL"))
fig.add_trace(go.Scatter(x=df_brechas.index, y = df_brechas["mep-equilibrio"], name = "MEP"))
fig.update_yaxes(tickformat = ".2%")
fig.update_layout(template = None, 
                  title_text = f"Brecha entre el tc de equilibrio y las distintas cotizaciones<br><sup>Datos hasta {ultima_fecha}")


In [11]:
# alphacast.datasets.dataset(37268).download_data("pandas")
alphacast.datasets.dataset(37268).upload_data_from_df(df, deleteMissingFromDB = True, onConflictUpdateDB = True, uploadIndex=True)
time.sleep(10)
alphacast.datasets.dataset(37447).upload_data_from_df(tc_read_df, deleteMissingFromDB = True, onConflictUpdateDB = True, uploadIndex=True)

b'{"id": 3320171, "status": "Requested", "createdAt": "2023-12-30T22:09:09.297573", "datasetId": 37447}'

In [12]:
time.sleep(3)
alphacast.datasets.dataset(37268).processes()


b'[{"id": 3320170, "datasetId": 37268, "status": "Processed", "statusDescription": "5486 values had conflict and were updated./nWARNING: More than 10%% of values changed. This may indicate a problem with the data./n16 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-12-30T22:08:57", "processedAt": "2023-12-30T22:09:00"}, {"id": 3252217, "datasetId": 37268, "status": "Processed", "statusDescription": "143 values had conflict and were updated./n21 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-12-14T14:32:33", "processedAt": "2023-12-14T14:32:36"}, {"id": 3168266, "datasetId": 37268, "status": "Processed", "statusDescription": "564 values had conflict and were updated./n43 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-11-23T19:15:15", "processedAt": "2023-11-23T19:15:19"}, {"id": 2992963, "datasetId": 37268, "status": "Processed", "

In [13]:
time.sleep(10)
alphacast.datasets.dataset(37447).processes()

b'[{"id": 3320171, "datasetId": 37447, "status": "Processed", "statusDescription": "0 values added to database./nDataset has not changed. Avoiding csv version upload.", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-12-30T22:09:09", "processedAt": "2023-12-30T22:09:11"}, {"id": 3252218, "datasetId": 37447, "status": "Processed", "statusDescription": "526 values had conflict and were updated./nWARNING: More than 10%% of values changed. This may indicate a problem with the data./n1 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-12-14T14:32:44", "processedAt": "2023-12-14T14:32:47"}, {"id": 3168267, "datasetId": 37447, "status": "Processed", "statusDescription": "522 values had conflict and were updated./nWARNING: More than 10%% of values changed. This may indicate a problem with the data./n2 values added to database./n", "deleteMissingFromDB": 1, "onConflictUpdateDB": 1, "createdAt": "2023-11-23T19:15:27", "proce

In [14]:
# time.sleep(10)
# alphacast.datasets.dataset(37268).download_data("pandas")